<a href="https://colab.research.google.com/github/matthewlai12/ECE-Lab/blob/main/ECE449Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Prompt Engineering Lab
## Tree of Thought Prompting with GPT4o-mini

Preliminaries: import libraries, read in the OpenAI API key and pass it to an OpenAI object constructor

In [ ]:
# Import libraries, set API key
from anytree import Node, RenderTree

import ASCIImaze
import os
import re

# This chunk of code reads an OpenAI API key from a chosen environment variable, which must be predefined.
# For my own dev, I'm using my personal API key. This will not be provided to others, you must use one you have
# access to.
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from openai import OpenAI

client = OpenAI(
    api_key=os.environ['SCOTT-OPENAI-KEY']
)


<b>First prompt:</b> Recognize the ASCII maze, and format it as a single-line string, with the different lines of the maze delimited by "|"

In [ ]:
extract_maze_system_prompt = ""

<b>Second prompt:</b> Format the 3x3 neighborhood in the same manner as the ASCII maze.

In [ ]:
generate_thought_context_system_prompt = ""

<b>Third prompt:</b> Based on the refomatted ASCII maze and neighborhood from prompts 1 and 2 above, choose the next move to make.

In [ ]:
generate_thought_system_prompt = ""


In [ ]:
# Possible models:
# gpt-3.5-turbo
# gpt-4o-mini

# get_completion is a helper method suggested in the DeepLearning.ai course on prompt engineering.
# I modified this version to use the OpenAI 1.0 implementation. The "client" object was passed an API key in the first code box.
def get_completion(system_prompt, user_prompt, my_model="gpt-4o-mini"):
    completion = client.chat.completions.create(
        model=my_model,
        messages=[
            {"role": "system",
             "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    return completion.choices[0].message


# -------------------------------------------------------------------------------------------------------------

# Utility function for displaying trees. Core code from the anytree package documentation.
def render_that_tree(mytree):
    for pre, _, node in RenderTree(mytree):
        print("%s%s" % (pre, node.name))

# ---------------------------------------------------------------------------------------------------------------

# Backtracking subroutine for Tree of Thought. Updates teh parent's state info to prune the dead-end branch we are
#  backtracking from (by setting that position in the stored 3x3 to '#').

def tree_backtrack(deadend_node, curr_move):

    curr_node = deadend_node.parent

    if deadend_node.move == 'D':
        temp = list(curr_node.name)
        temp[9] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'U':
        temp = list(curr_node.name)
        temp[1] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'L':
        temp = list(curr_node.name)
        temp[4] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'R':
        temp = list(curr_node.name)
        temp[6] = '#'
        curr_node.name = ''.join(temp)

    else:
        print("Illegal move in backtracking")
        return None

    return curr_node

# ----------------------------------------------------------------------------------------------------------------

# Add node subroutine for Tree of Thought. Pretty short, but putting it here makes teh main loop cleaner.
#  This also pushes the tree updates out to subroutines, which is common and familiar. Leaves just teh LLM and game
#  logic in the main loop, which I think is good.
def add_node(curr_node, LLM_move_cleaned):
    # New node is a child of the current node. State has to be determined on next iteration. The move to reach this
    #  node from its parent is recorded as the "move" attribute. The move to reach its parent from this node
    #  is recorded as the "backtrack" attribute.

    # Have to determine the backtrack attribute by reversing LLM_cleaned_move. Another elif tree here.
    if LLM_move_cleaned.find('D') >= 0:
        rev = 'U'

    elif LLM_move_cleaned.find('U') >= 0:
        rev = 'D'

    elif LLM_move_cleaned.find('L') >= 0:
        rev = 'R'

    elif LLM_move_cleaned.find('R') >= 0:
        rev = 'L'

    else:
        print("Not a legal move")
        return None

    temp = Node(parent=curr_node, name="", move=LLM_move_cleaned, backtrack=rev)

    return temp


# ----------------------------------------------------------------------------------------------------------

def evaluate_thought(curr_local):
    # Pre: curr_local is the 3x3 neighborhood for the current location in the maze.
    # Format ###|#*#|###
    # Post: none
    # Returns: List of valid moves, each a 1-character string from {U, D, L, R}

    # Create output list, convert input string to a list.

    # Debug
    print(f"evaluate thought recevied {curr_local}")
    # Debug

    valid_moves = []
    maze_neigh = list(curr_local)
    # print(f"Local neighborhood string is {curr_local}")
    # print(f"Converted to a List it is {maze_neigh}")

    if maze_neigh[1] == ' ' or maze_neigh[1] == 'S' or maze_neigh[1] == 'E':
        valid_moves.append("U")
        # print("found U")

    if maze_neigh[4] == ' ' or maze_neigh[4] == 'S' or maze_neigh[4] == 'E':
        valid_moves.append("L")
        # print("found L")

    if maze_neigh[6] == ' ' or maze_neigh[6] == 'S' or maze_neigh[6] == 'E':
        valid_moves.append("R")
        # print("found R")

    if maze_neigh[9] == ' ' or maze_neigh[9] == 'S' or maze_neigh[9] == 'E':
        valid_moves.append("D")
        # print("found D")

    return valid_moves

# ----------------------------------------------------------------------------------------------------------

# Guards for 1st iteration, context request.
ready_for_input = False
need_context_update = True
mazeTreeStart = Node(name="", move="", backtrack="")
curr_node = mazeTreeStart
# Debug
counter = 0

# The maze game loop is contained in the maze.solve() method. Call the constructor, and then enter a for loop
#  to receive outputs from maze.solve(), and respond with our inputs.
maze = ASCIImaze.Maze("maze_map1", show_moves=False, show_coords=False)
for output in maze.solve():
    if (not ready_for_input):
        # On the first iteration, we get the maze to solve. Bring it into the LLM, then goto next iteration; that
        #  will be the first opportunity to pass an input to the ASCIImaze solver.
        # print(output)
        # Going to pull the maze into the LLM's chat history now.
        user_prompt=f"""```{output}```"""
        maze_to_solve = get_completion(extract_maze_system_prompt, user_prompt)
        #print(maze_to_solve.content)
        ready_for_input = True
        continue

    if (need_context_update):
        # Grab context by passing '3' to ASCIIMaze. Isolated so can use stored context instead.
        # Must go to next iteration b/c only then will 'output' be updated with the 3x3 neighborhood.
        maze.get_user_move('3')
        need_context_update = False
        continue

    if "Maze solved!" in output:
        # Victory!
        print(output)
        break


    # Pull in the current 3x3 neighborhood, IF the current node does not already have a state. Otherwise, this is a
    #  backtracking situation.

    # Debug
    print(f"Output reads: {output}")
    if curr_node.name == "":
        # Case for a node not previously visited.
        user_prompt = f"```{output}```"
        neigh = get_completion(generate_thought_context_system_prompt, user_prompt)
        curr_node.name = neigh.content


    # Evaluate the current position; this is actually evaluating the through generated in the previous round.
    # Possible results are {{valid moves}, <just the backtrack direction>}
    # Backtrack if so indicated, generate a new thought if there are other valid moves.
    # The ASCIImaze.maze.solve() method, the main game loop, outputs a message when you correctly solve the maze, you
    #  don't need to figure out if the LLM has won the game. Just check for the victory message. Done above.


    print(f"Maze neighborhood is {curr_node.name}")

    valid_moves_cleaned = evaluate_thought(curr_node.name)


    print(f"Valid moves are {valid_moves_cleaned}")



    if len(valid_moves_cleaned) == 1 and valid_moves_cleaned[0] == curr_node.backtrack:
        # The only move detected is the backtrack direction, so follow it, both in the maze and the tree.
        maze.get_user_move(curr_node.backtrack)
        curr_node = tree_backtrack(curr_node, curr_node.backtrack)

        # Do not update context, the state in curr_node has already been updated. BUT this blows up an assumption :(
        need_context_update = False

        # The valid moves from the last eval step are no longer correct. Go to the next iteration, get a new move set.
        continue

    # Generate a thought
    # The LLM chooses a next move, which will be a child of the current node
    # Again, inner monologues are used to recreate the 2D mazes.
    user_prompt = f"""```{maze_to_solve.content}, {curr_node.name}```"""

    # Adding the backtrack move to thought generation. Remember to deal with root node (no parent).
    valid_moves_temp = ", ".join(valid_moves_cleaned)

    # Initial value for LLM_move, for loop entry
    LLM_move_cleaned = ['Z']

    # Loop to check for valid move.
    # LLM_move_cleaned should be a single-element list. If that element is not part of the valid move set, try again.
    while valid_moves_temp.find(LLM_move_cleaned[0]) < 0:

        print(f"Moves going to LLM are {valid_moves_temp}")

        if curr_node.parent != None:
            LLM_move = get_completion(generate_thought_system_prompt.format(vmove=valid_moves_temp,
                                                                    backmove=curr_node.backtrack), user_prompt)
        else:
            LLM_move = get_completion(generate_thought_system_prompt.format(vmove=valid_moves_temp,
                                                                        backmove=None), user_prompt)

        LLM_move_temp = re.sub('\&\&\&.+\&\&\&', '', LLM_move.content, flags=re.DOTALL)
        # Again, another regex needed.
        LLM_move_cleaned = re.findall("[A-Z]", LLM_move_temp)
        print(f"I choose move {LLM_move_cleaned[0]}")
    # Error check
    if len(LLM_move_cleaned) > 1:
        print("Extra stuff in the generated thought, aborting")
        break


    # Make the chosen move, and add the corresponding node to the tree.
    maze.get_user_move(LLM_move_cleaned[0])
    curr_node = add_node(curr_node, LLM_move_cleaned[0])
    if curr_node is None:
        print("You goofed.")
        print(LLM_move_cleaned)
    # Debug
    counter = counter + 1
    print(f"I choose move {LLM_move_cleaned}")
    print(f"This is move {counter}")

    # Need the next 3x3 neighborhood.
    need_context_update = True

print("I finished")



Output reads: Here is a 3x3 of your current position:
###
#* 
# #
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
Maze neighborhood is ###|#* |# #
evaluate thought recevied ###|#* |# #
Valid moves are ['R', 'D']
Moves going to LLM are R, D
I choose move R
I choose move ['R']
This is move 1
Output reads: Here is a 3x3 of your current position:
###
S* 
 ##
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
Maze neighborhood is ###|S* | ##
evaluate thought recevied ###|S* | ##
Valid moves are ['L', 'R']
Moves going to LLM are L, R
I choose move R
I choose move ['R']
This is move 2
Output reads: Here is a 3x3 of your current position:
###
 * 
###
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
Maze neighborhood is ###| * |###
evaluate thought recevied ###| * |###
Valid moves are ['L', 'R']
Moves going to LLM are L, R
I choose move R
I choose move ['R']
This is move 3
Output reads: Here is a 3x3 of 